In [174]:
from pathlib import Path

import numpy as np
import open3d as o3d
from scipy.spatial import Delaunay

In [175]:
support_path = Path('../out/support.stl')
support_mesh = o3d.io.read_triangle_mesh(support_path)
print(support_mesh)
support_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([support_mesh])

TriangleMesh with 160 points and 88 triangles.


In [176]:
verts = np.asarray(support_mesh.vertices)
tris = np.asarray(support_mesh.triangles)

# for axis in verts:
#     print((np.max(axis[0]), np.max(axis[1]), np.max(axis[2])))


verts

array([[ 122.        ,   -2.5       ,  122.        ],
       [ 120.58578491,   -3.08578634,  120.58578491],
       [ 122.        ,   -2.5       , -122.        ],
       [ 120.58578491,   -3.08578634, -120.58578491],
       [ 120.58578491,   -3.08578634, -120.58578491],
       [ 120.58578491,   -3.08578634,  120.58578491],
       [ 120.15224457,   -3.73463297,  120.15224457],
       [ 120.15224457,   -3.73463297, -120.15224457],
       [ 120.15224457,   -3.73463297, -120.15224457],
       [ 120.15224457,   -3.73463297,  120.15224457],
       [ 120.        ,   -4.5       , -120.        ],
       [ 120.        ,   -4.5       ,  120.        ],
       [ 120.        ,   -4.5       , -120.        ],
       [-120.        ,   -4.5       , -120.        ],
       [ 120.15224457,   -3.73463297, -120.15224457],
       [-120.15224457,   -3.73463297, -120.15224457],
       [ 120.15224457,   -3.73463297, -120.15224457],
       [-120.15224457,   -3.73463297, -120.15224457],
       [-120.58578491,   -3.

In [177]:
x_min = np.min(verts[:, 0])
x_max = np.max(verts[:, 0])
y_min = np.min(verts[:, 1])
y_max = np.max(verts[:, 1])
z_min = np.min(verts[:, 2])
z_max = np.max(verts[:, 2])
print(x_min, x_max, y_min, y_max, z_min, z_max)

-125.0 125.0 -7.5 7.5 -125.0 125.0


In [178]:
top_edge_verts = []
bottom_edge_verts = []

for vert in verts:
    if vert[1] == y_max:
        top_edge_verts.append(vert)
    elif vert[1] == y_min:
        bottom_edge_verts.append(vert)

top_edge_verts = np.array(top_edge_verts)
bottom_edge_verts = np.array(bottom_edge_verts)
# print(top_edge_verts)
# print(bottom_edge_verts)

In [179]:
from scipy.stats import norm

n = 240
mu1, sigma1 = 0, 1
mu2, sigma2 = 0, 1

x = 3 * sigma1
y = 3 * sigma2


# x = np.linspace(-xlen, xlen, n)
# y = np.linspace(-ylen, ylen, n)

x = np.linspace(-x, x, n)
y = np.linspace(-y, y, n)

X, Y = np.meshgrid(x, y)

Z = norm.pdf(X, mu1, sigma1) * norm.pdf(Y, mu2,
                                        sigma2) * 10
top_surface = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

In [180]:
# for vert in top_surface:
#     if vert[0] ==x_min or vert[0] == self._x_max:
#         vert[2] = self.sidewall_height
#     elif vert[1] == self._y_min or vert[1] == self._y_max:
#         vert[2] = self.sidewall_height
top_surface

array([[-3.00000000e+00, -3.00000000e+00,  1.96412803e-04],
       [-2.97489540e+00, -3.00000000e+00,  2.11709970e-04],
       [-2.94979079e+00, -3.00000000e+00,  2.28054747e-04],
       ...,
       [ 2.94979079e+00,  3.00000000e+00,  2.28054747e-04],
       [ 2.97489540e+00,  3.00000000e+00,  2.11709970e-04],
       [ 3.00000000e+00,  3.00000000e+00,  1.96412803e-04]],
      shape=(57600, 3))

In [181]:
top_tri = Delaunay(top_surface[:, :2]).simplices

top_mesh = o3d.geometry.TriangleMesh()
top_mesh.triangles = o3d.utility.Vector3iVector(top_tri)
top_mesh.vertices = o3d.utility.Vector3dVector(top_surface)


# next steps:
# 1. create a normal surface which goes from x[-120, 120], y[-120,120], with a std_dev which makes the surface look like a scaled up standard normal
# 2. rotate the surface so its in the same orientation as the support mesh
# 3. zeroize the edges of the normal surface
# 3. merge the two meshes into one

In [182]:
o3d.visualization.draw_geometries([support_mesh + top_mesh])